In [1]:
import pandas as pd
from text_processing import custom_tokenizer, custom_tokenizer_bow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
# warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jacobhansen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
url='https://raw.githubusercontent.com/jacob-hansen/NLP_in_EHR_2022/910d9f0fcfeab083dff53ea2e2969c175cc816a0/train.csv'
train_df = pd.read_csv(url)
# apply the custom tokenizer to the dataframe
train_df['tokenized'] = train_df['X_train'].apply(custom_tokenizer_bow)
# create vocabulary of size 1000
vocab_size = 1000
# create a tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_df['tokenized'])
# encode training data set
data = tokenizer.texts_to_sequences(train_df['tokenized'])
# pad sequences
data = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post')

/Users/jacobhansen/opt/anaconda3/envs/dev2022/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['arm', -1.173, 'areas']


NameError: name 'breaksys' is not defined

# Training Transformer Model

In [20]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), 
             Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
        
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [21]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = np.max([len(x) for x in data])  # Maximum number of words in a sentence
vocab_size = 1000 # size of the vocabulary

inputs = Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(2, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

In [22]:
X = np.array(data)
y = np.array(train_df['y_train'])

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# split the train data into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [24]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, 
                    batch_size=64, epochs=100, 
                    validation_data=(X_val, y_val)
                   )

Epoch 1/100
100/100 [==============================] - 3s 24ms/step - loss: 0.5142 - accuracy: 0.7965 - val_loss: 0.5068 - val_accuracy: 0.8012
Epoch 2/100
100/100 [==============================] - 2s 24ms/step - loss: 0.5109 - accuracy: 0.7975 - val_loss: 0.5004 - val_accuracy: 0.8012
Epoch 3/100
100/100 [==============================] - 2s 22ms/step - loss: 0.5020 - accuracy: 0.8035 - val_loss: 0.5043 - val_accuracy: 0.8012
Epoch 4/100
100/100 [==============================] - 3s 26ms/step - loss: 0.4988 - accuracy: 0.8030 - val_loss: 0.5001 - val_accuracy: 0.8012
Epoch 5/100
100/100 [==============================] - 2s 23ms/step - loss: 0.4976 - accuracy: 0.8030 - val_loss: 0.5016 - val_accuracy: 0.8012
Epoch 6/100
100/100 [==============================] - 2s 23ms/step - loss: 0.4979 - accuracy: 0.8024 - val_loss: 0.5035 - val_accuracy: 0.8012
Epoch 7/100
100/100 [==============================] - 2s 22ms/step - loss: 0.5035 - accuracy: 0.7966 - val_loss: 0.5016 - val_accuracy:

KeyboardInterrupt: 